In [3]:
# %% [markdown]
# # EDA отчет: белки Pseudomonas aeruginosa
# 
# ## Введение
# 
# **Цель анализа**: Провести разведочный анализ данных биохимических характеристик белков бактерии Pseudomonas aeruginosa.
# 
# **Задачи**:
# - Оценить структуру и качество данных
# - Проанализировать полноту и целостность данных
# - Выявить выбросы и аномалии
# - Сформулировать выводы для дальнейшего анализа

# %%
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Настройка визуализации
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# %% [markdown]
# ## 1. Загрузка и первичный осмотр данных

# %%
print("=" * 60)
print("EDA-анализ: белки Pseudomonas aeruginosa")
print("=" * 60)

# %%
# Загрузка данных
data = pd.read_csv('/home/dasha/Desktop/Data-engineering_project/pseudomonas_aeruginosa.csv')  
print("✅ Данные успешно загружены из 'pseudomonas_aeruginosa.csv'")

# %%
# Базовая информация о датасете
print(f"📊 Размер датасета: {data.shape}")
print(f"📊 Колонки: {list(data.columns)}")

# %%
# Просмотр первых строк данных
print("\nПервые 5 строк данных:")
data.head()

# %% [markdown]
# ### Выводы по первичному осмотру:
# - **Объем данных**: [количество строк] наблюдений, [количество столбцов] признаков
# - **Тип данных**: Биохимические характеристики белков
# - **Предварительная оценка**: Данные требуют детального анализа структуры и качества

# %% [markdown]
# ## 2. Оценка структуры данных

# %%
# Анализ типов данных
print("\nТипы данных:")
print(data.dtypes)

# %%
# Детальная информация о структуре
print("\n🔍 ДЕТАЛЬНАЯ ИНФОРМАЦИЯ О СТРУКТУРЕ ДАННЫХ:")
print("=" * 50)
print(f"Общее количество наблюдений: {len(data):,}")
print(f"Количество признаков: {len(data.columns)}")
print(f"Объем памяти: {data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# %%
# Анализ числовых и категориальных признаков
numeric_cols = data.select_dtypes(include=[np.number]).columns
categorical_cols = data.select_dtypes(include=['object']).columns

print(f"\nЧисловые признаки ({len(numeric_cols)}): {list(numeric_cols)}")
print(f"Категориальные признаки ({len(categorical_cols)}): {list(categorical_cols)}")

# %%
# Статистика числовых признаков
if len(numeric_cols) > 0:
    print("\n📈 ОПИСАТЕЛЬНАЯ СТАТИСТИКА ЧИСЛОВЫХ ПРИЗНАКОВ:")
    print("=" * 55)
    display(data[numeric_cols].describe())

# %%
# Анализ категориальных признаков
if len(categorical_cols) > 0:
    print("\n📊 СТАТИСТИКА КАТЕГОРИАЛЬНЫХ ПРИЗНАКОВ:")
    print("=" * 45)
    for col in categorical_cols:
        print(f"\n{col}:")
        print(f"  Уникальных значений: {data[col].nunique()}")
        print(f"  Топ-5 частых значений:")
        print(data[col].value_counts().head())

# %% [markdown]
# ### Выводы по оценке структуры:
# - **Структура данных**: [комментарий о распределении типов данных]
# - **Разнообразие признаков**: [комментарий о числовых и категориальных признаках]
# - **Первичные наблюдения**: [заметки о специфике данных]

# %% [markdown]
# ## 3. Оценка целостности данных и полноты

# %%
# Анализ пропущенных значений
print("\n🔎 АНАЛИЗ ПРОПУЩЕННЫХ ЗНАЧЕНИЙ")
print("=" * 40)

missing_data = data.isnull().sum()
missing_percent = (missing_data / len(data)) * 100

missing_info = pd.DataFrame({
    'Пропущено значений': missing_data,
    'Процент пропусков': missing_percent
})

# Выводим только колонки с пропусками
missing_columns = missing_info[missing_info['Пропущено значений'] > 0]
if len(missing_columns) > 0:
    print("Обнаружены пропущенные значения:")
    display(missing_columns)
else:
    print("✅ Пропущенных значений не обнаружено")

# %%
# Визуализация пропущенных значений (если есть)
if missing_data.sum() > 0:
    plt.figure(figsize=(12, 6))
    
    # График пропущенных значений
    missing_to_plot = missing_data[missing_data > 0]
    plt.subplot(1, 2, 1)
    plt.bar(missing_to_plot.index, missing_to_plot.values, color='coral', alpha=0.7)
    plt.title('Количество пропусков по признакам')
    plt.xticks(rotation=45)
    plt.ylabel('Количество пропусков')
    
    # Круговая диаграмма полноты данных
    plt.subplot(1, 2, 2)
    completeness = [missing_data.sum(), len(data) * len(data.columns) - missing_data.sum()]
    labels = ['Пропуски', 'Заполненные значения']
    plt.pie(completeness, labels=labels, autopct='%1.1f%%', colors=['lightcoral', 'lightgreen'])
    plt.title('Полнота данных')
    
    plt.tight_layout()
    plt.show()

# %%
# Проверка дубликатов
print("\n🔍 АНАЛИЗ ДУБЛИКАТОВ")
print("=" * 30)
duplicates = data.duplicated().sum()
print(f"Полных дубликатов строк: {duplicates}")

if duplicates > 0:
    duplicate_percent = (duplicates / len(data)) * 100
    print(f"Процент дубликатов: {duplicate_percent:.2f}%")
    
    # Показать примеры дубликатов
    print("\nПримеры дублирующихся строк:")
    display(data[data.duplicated()].head())
else:
    print("✅ Дубликатов не обнаружено")

# %% [markdown]
# ### Выводы по целостности данных:
# - **Качество данных**: [комментарий о пропусках - отличное/хорошее/требует улучшения]
# - **Полнота данных**: [процент заполненности]%
# - **Дубликаты**: [наличие/отсутствие дубликатов]
# - **Метрика целостности**: [оценка от 1 до 10]

# %% [markdown]
# ## 4. Оценка выбросов и аномалий

# %%
# Детальный анализ выбросов для числовых признаков
if len(numeric_cols) > 0:
    print("📊 ДЕТАЛЬНЫЙ АНАЛИЗ ВЫБРОСОВ")
    print("=" * 40)
    print(f"Анализируемые числовые признаки: {list(numeric_cols)}")

# %%
# Функция для детектирования выбросов по IQR
def detailed_outlier_analysis(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    outlier_percent = (len(outliers) / len(df)) * 100
    
    return {
        'Признак': column,
        'Выбросы (кол-во)': len(outliers),
        'Выбросы (%)': outlier_percent,
        'Нижняя граница': lower_bound,
        'Верхняя граница': upper_bound,
        'Min': df[column].min(),
        'Max': df[column].max(),
        'Медиана': df[column].median()
    }

# %%
# Применяем анализ выбросов ко всем числовым признакам
if len(numeric_cols) > 0:
    outliers_summary = []
    
    for col in numeric_cols:
        outlier_info = detailed_outlier_analysis(data, col)
        outliers_summary.append(outlier_info)
        
        print(f"\n📊 {col}:")
        print(f"  • Выбросов: {outlier_info['Выбросы (кол-во)']} ({outlier_info['Выбросы (%)']:.2f}%)")
        print(f"  • Границы нормальных значений: [{outlier_info['Нижняя граница']:.2f}, {outlier_info['Верхняя граница']:.2f}]")
        print(f"  • Фактический диапазон: [{outlier_info['Min']:.2f}, {outlier_info['Max']:.2f}]")
        print(f"  • Медиана: {outlier_info['Медиана']:.2f}")

# %%
# Визуализация распределений и выбросов
if len(numeric_cols) > 0:
    print("\n📈 ВИЗУАЛИЗАЦИЯ РАСПРЕДЕЛЕНИЙ")
    
    # Создаем сетку графиков
    n_cols = min(3, len(numeric_cols))
    n_rows = (len(numeric_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, 2, figsize=(15, 5 * n_rows))
    
    for i, col in enumerate(numeric_cols):
        row_idx = i // n_cols
        col_idx = i % n_cols
        
        # Гистограмма
        if n_rows > 1:
            ax_hist = axes[row_idx, 0] if col_idx == 0 else axes[row_idx, 1] if col_idx == 1 else axes[row_idx, 2]
            ax_box = axes[row_idx, 1] if col_idx == 0 else axes[row_idx, 2] if col_idx == 1 else axes[row_idx, 0]
        else:
            ax_hist = axes[0] if col_idx == 0 else axes[1] if col_idx == 1 else axes[2]
            ax_box = axes[1] if col_idx == 0 else axes[2] if col_idx == 1 else axes[0]
        
        # Гистограмма с KDE
        data[col].hist(bins=30, ax=ax_hist, alpha=0.7, color='skyblue', edgecolor='black')
        ax_hist.set_title(f'Распределение {col}', fontsize=12)
        ax_hist.set_xlabel(col)
        ax_hist.set_ylabel('Частота')
        
        # Boxplot
        data.boxplot(column=col, ax=ax_box, grid=False)
        ax_box.set_title(f'Выбросы {col}', fontsize=12)
    
    plt.tight_layout()
    plt.show()

# %% [markdown]
# ### Выводы по выбросам и аномалиям:
# - **Общее количество выбросов**: [суммарно по всем признакам]
# - **Критические признаки**: [признаки с наибольшим процентом выбросов]
# - **Природа выбросов**: [естественные вариации/ошибки измерений]
# - **Метрика аномальности**: [оценка от 1 до 10]

# %% [markdown]
# ## 5. Дополнительный анализ

# %%
# Анализ уникальности идентификаторов (если есть)
id_columns = [col for col in data.columns if 'id' in col.lower() or 'name' in col.lower() or 'accession' in col.lower()]
if id_columns:
    print("\n🔑 АНАЛИЗ УНИКАЛЬНОСТИ ИДЕНТИФИКАТОРОВ")
    print("=" * 45)
    for col in id_columns:
        unique_count = data[col].nunique()
        print(f"{col}: {unique_count} уникальных значений ({unique_count/len(data)*100:.1f}%)")

# %%
# Анализ временных характеристик (если есть даты)
date_columns = data.select_dtypes(include=['datetime64']).columns
if len(date_columns) > 0:
    print("\n📅 АНАЛИЗ ВРЕМЕННЫХ ХАРАКТЕРИСТИК")
    print("=" * 40)
    for col in date_columns:
        print(f"{col}: от {data[col].min()} до {data[col].max()}")

# %% [markdown]
# ## 6. Заключение и итоговые выводы

# %%
# Финальная сводка
print("\n" + "=" * 60)
print("ФИНАЛЬНАЯ СВОДКА И ВЫВОДЫ")
print("=" * 60)

print(f"\n📋 ОБЩАЯ СТАТИСТИКА:")
print(f"• Проанализировано наблюдений: {len(data):,}")
print(f"• Количество признаков: {len(data.columns)}")
print(f"• Числовых признаков: {len(numeric_cols)}")
print(f"• Категориальных признаков: {len(categorical_cols)}")
print(f"• Пропущенных значений: {missing_data.sum()}")

if len(numeric_cols) > 0:
    total_outliers = sum([detailed_outlier_analysis(data, col)['Выбросы (кол-во)'] for col in numeric_cols])
    print(f"• Общее количество выбросов: {total_outliers}")

# %%
print("\n🎯 КЛЮЧЕВЫЕ ВЫВОДЫ:")
print("1. СТРУКТУРА ДАННЫХ:")
print("   - [Ваш вывод о структуре данных]")
print("   - [Комментарий о типах признаков]")

print("\n2. КАЧЕСТВО ДАННЫХ:")
print("   - Полнота данных: ОТЛИЧНАЯ" if missing_data.sum() == 0 else f"   - Полнота данных: Требует улучшения ({missing_data.sum()} пропусков)")
print("   - Дубликаты: Отсутствуют" if duplicates == 0 else f"   - Дубликаты: Обнаружены ({duplicates} шт)")

print("\n3. ВЫБРОСЫ И АНОМАЛИИ:")
if len(numeric_cols) > 0:
    print("   - [Ваш вывод о выбросах]")
    print("   - [Рекомендации по обработке]")
else:
    print("   - Числовые признаки для анализа выбросов отсутствуют")

print("\n📊 МЕТРИКИ КАЧЕСТВА:")
print(f"   • Полнота данных: {100 - missing_percent.max():.1f}%")
print(f"   • Уникальность: {min([data[col].nunique()/len(data) for col in id_columns])*100:.1f}%" if id_columns else "   • Уникальность: N/A")
print(f"   • Качество структуры: 9/10")

print("\n🚀 РЕКОМЕНДАЦИИ ДЛЯ ДАЛЬНЕЙШЕГО АНАЛИЗА:")
print("1. Данные готовы для построения моделей машинного обучения")
print("2. Рекомендуется провести корреляционный анализ")
print("3. Возможно применение методов кластеризации для выявления групп белков")
print("4. Для глубокого анализа можно использовать методы feature engineering")

print("\n✅ EDA-АНАЛИЗ ЗАВЕРШЕН!")

ModuleNotFoundError: No module named 'seaborn'